<a href="https://colab.research.google.com/github/ivanleech/llm_evaluator/blob/main/llm_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Trulens to evaluate LLM/RAG 🔎

📓 This notebook explores Advanced RAG methods using Direct Query as baseline, and compares them to sentence-window and automerging-retrieval methods.

To evaluate the responses and context retrieved from various methods, we use TruLens🐙 to explore a Triad of metrics, context relevance(context to query), answer relevance(answer to query) and groundedness(answer to context).

Instead of using OpenAI, we use Ollama 🦙 to install various models locally, allowing users without access to OpenAI to run these notebook without any issue. This notebook is recommend to run using Google Colab using the free T4 GPU compute. 🖥️


## 01 Setup

In [1]:
%%capture
!git clone https://github.com/ivanleech/llm_evaluator.git
!pip install -r llm_evaluator/requirements.txt

In [2]:
!curl https://ollama.ai/install.sh | sh

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100  8422    0  8422    0     0  26456      0 --:--:-- --:--:-- --:--:-- 26484
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 0.0.0.0:11434.
>>> Install complete. Run "ollama" from the command line.
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
import urllib.request
from llama_index import Document
from llama_index import SimpleDirectoryReader

# Downloads document and store as pdf locally
url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
file = 'attention.pdf'
urllib.request.urlretrieve(url, file)

# Create llama_index document which will be used to create index
# The created index will provide context and used to answer questions using RAG later
documents = SimpleDirectoryReader(input_files=[file]).load_data()
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [4]:
import os
from time import sleep
from subprocess import Popen
from llama_index.embeddings import HuggingFaceEmbedding

# Pull embedding llm and reranking llm from HuggingFace
# model = 'dolphin-phi'
model = 'llama2'
base_url = 'http://localhost:11434'
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")
reranker_model = "BAAI/bge-reranker-base"

# Pulls model from Ollama. Will be used as evaluating llm and generating llm
p = Popen(["ollama", "serve"])  # something long running
sleep(1)
os.system(f'ollama pull {model}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

0

In [5]:
from langchain.llms import Ollama
from trulens_eval import LiteLLM
import litellm
litellm.set_verbose=False

# 2 objects are created from the same model, 1 for generating answers, 1 for evaluating the RAG performance

# Used by llama_index ServiceContext as generating llm
ollama = Ollama(base_url=base_url, model=model)

# Used by Trulens as evaluation llm
ollama_provider = LiteLLM(model_engine=f"ollama/{model}", api_base=base_url)

## 02 Advanced RAG

Retrieval Augmented Generation


### Direct Query Engine

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext

# Creates ServiceContext, which is a bundler to hold generating llm and embedding llm
service_context = ServiceContext.from_defaults(llm=ollama, embed_model=embed_model)

# VectorStoreIndex converts text from earlier document into embeddings using embedding llm
# query_engine is created from the index, and is able to answer questions with context from the document
index = VectorStoreIndex.from_documents([document], service_context=service_context)
query_engine = index.as_query_engine()

## Sentence-window retrieval

In [7]:
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import StorageContext
from llama_index import load_index_from_storage

# SentenceWindowNodeParser takes in parameters used to decide how the sentence window RAG is
node_parser = SentenceWindowNodeParser.from_defaults(window_size=3, window_metadata_key="window", original_text_metadata_key="original_text")
sentence_context = ServiceContext.from_defaults(llm=ollama, embed_model=embed_model, node_parser=node_parser)

# Creates index if not found locally and save to local dir, load from local otherwise
save_dir = 'sentence_index'
if not os.path.exists(save_dir):
    sentence_index = VectorStoreIndex.from_documents([document], service_context=sentence_context)
    sentence_index.storage_context.persist(persist_dir=save_dir)
else:
    sentence_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=save_dir), service_context=sentence_context)

In [8]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank

rerank_top_n=2
similarity_top_k=6

# From index, retrieve top k documents most simlar to user query (i.e retrieve top 6 most similar documents)
# From retrieved documents, rerank and get top n most relavant results to be used as context for RAG (i.e rerank and get top 2 most relavant results)
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
rerank = SentenceTransformerRerank(top_n=rerank_top_n, model=reranker_model)

sentence_window_engine = sentence_index.as_query_engine(similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank])

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

## Automerging Retrieval

In [9]:
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes


chunk_sizes = [2048, 512, 128]
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
nodes = node_parser.get_nodes_from_documents(documents)
leaf_nodes = get_leaf_nodes(nodes)
merging_context = ServiceContext.from_defaults(llm=ollama, embed_model=embed_model)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# Creates index if not found locally and save to local dir, load from local otherwise
save_dir = 'merging_index'
if not os.path.exists(save_dir):
    automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=merging_context)
    automerging_index.storage_context.persist(persist_dir=save_dir)
else:
    automerging_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=save_dir), service_context=merging_context,)

In [10]:
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

# From leaf nodes, if insufficient context is found, parents nodes will be retrieved to provide context.
# This will be repeated until sufficient context is found
base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
retriever = AutoMergingRetriever(base_retriever, automerging_index.storage_context, verbose=True)
rerank = SentenceTransformerRerank(top_n=rerank_top_n, model=reranker_model)
auto_merging_engine = RetrieverQueryEngine.from_args(retriever, service_context=merging_context, node_postprocessors=[rerank])

## 03 Use Trulens to evaluate model

In [11]:
import numpy as np
from trulens_eval.feedback import Groundedness
from trulens_eval import Feedback, TruLlama

qa_relevance = (Feedback(ollama_provider.relevance_with_cot_reasons, name="Answer Relevance")
              .on_input_output())

qs_relevance = (Feedback(ollama_provider.relevance_with_cot_reasons, name = "Context Relevance")
              .on_input()
              .on(TruLlama.select_source_nodes().node.text)
              .aggregate(np.mean))

grounded = Groundedness(groundedness_provider=ollama_provider)

groundedness = (Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
              .on(TruLlama.select_source_nodes().node.text)
              .on_output()
              .aggregate(grounded.grounded_statements_aggregator))

feedbacks = [qa_relevance, qs_relevance, groundedness]

eval_questions = ['What is the paper about?', 'What is attention in context of the paper?', 'What are the benefits of using Transformer?']

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [12]:
tru_recorder = TruLlama(query_engine, app_id='Direct Query Engine', feedbacks=feedbacks)
for question in eval_questions:
    with tru_recorder as recording:
        response = query_engine.query(question)
        print(question)
        print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


What is the paper about?
The paper "Attention Is All You Need" by Ashish Vaswani et al. discusses the Transformer architecture, a new approach to neural machine translation that replaces traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs) with attention mechanisms. The paper shows that this attention-based architecture can be trained significantly faster than previous models and achieves state-of-the-art results on the WMT 2014 English-to-German and English-to-French translation tasks.

The paper introduces several key innovations:

1. Multi-head attention: Instead of performing a single attention function with dmodel-dimensional keys, values, and queries, the Transformer performs multiple attention functions in parallel, each with their own learned linear projection. This allows the model to jointly attend to information from different representation subspaces at different positions.
2. Residual connections: The paper uses residual connections to faci

In [13]:
tru_recorder_sentence_window = TruLlama(sentence_window_engine, app_id='Sentence Window Query Engine', feedbacks=feedbacks)
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(response)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


What is the paper about?
Based on the provided context information, it seems that the paper is focused on developing a deep reinforcement learning model for abstractive summarization. The authors propose a new approach that leverages the power of deep neural networks and reinforcement learning to learn a mapping between input sentences and their corresponding summaries. The key idea is to use an embedding space to represent the input sentences and the summaries, and then use a reinforcement learning agent to learn the mapping between these two spaces. The paper also cites previous work on using output embeddings to improve language models, which suggests that the proposed approach may build upon existing research in this area. Overall, the paper appears to be focused on exploring new ways to use deep learning and reinforcement learning for summarization tasks.


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


What is attention in context of the paper?
In the context of the paper, attention refers to a mechanism used in various models to relate different positions of a single sequence in order to compute a representation of the sequence. Specifically, self-attention is an attention mechanism that relates different positions of a single sequence, while end-to-end memory networks are based on a recurrent attention mechanism instead of sequence-aligned recurrence. The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution.
What are the benefits of using Transformer?
The Transformer model offers several benefits over traditional neural sequence transduction models. Here are some of the key advantages of using Transformer:

1. **Parallelization**: Transformer can be parallelized more easily than RNNs, which makes it faster and more scalable for large-scale tasks. This is b

In [14]:
tru_recorder_automerging = TruLlama(auto_merging_engine, app_id='Automerging Query Engine', feedbacks=feedbacks)
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = auto_merging_engine.query(question)
        print(question)
        print(response)

What is the paper about?
The paper "Attention Is All You Need" by Ashish Vaswani et al. discusses a new neural network architecture for machine translation called the Transformer model. The Transformer model relies entirely on self-attention mechanisms, rather than traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs), to process input sequences.

The paper presents experimental results showing that the Transformer model outperforms state-of-the-art RNN and CNN models in machine translation tasks. The authors also analyze the attention mechanisms used by the Transformer model, showing how they allow the model to efficiently focus on different parts of the input sequence during processing.

Overall, the paper demonstrates the effectiveness of the Transformer model for machine translation tasks and highlights the importance of attention mechanisms in neural network architectures.
What is attention in context of the paper?
In the context of the paper, attent

In [15]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.850000,0.75,0.45,38.333333,0.0
Direct Query Engine,0.783333,1.00,0.60,38.333333,0.0
Sentence Window Query Engine,0.700000,1.00,0.60,38.333333,0.0


In [16]:
# tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

In [17]:
!curl ipecho.net/plain

34.30.62.33

In [1]:
try:
  tru.stop_dashboard()
except:
  pass
sleep(5)
# click the url provided and key in above ip in the box
tru.run_dashboard(port=8501)